# --- Day 16: The Floor Will Be Lava ---

## --- Part One ---

With the beam of light completely focused **somewhere**, the reindeer leads you deeper still into the Lava Production Facility. At some point, you realize that the steel facility walls have been replaced with cave, and the doorways are just cave, and the floor is cave, and you're pretty sure this is actually just a giant cave.

Finally, as you approach what must be the heart of the mountain, you see a bright light in a cavern up ahead. There, you discover that the beam of light you so carefully focused is emerging from the cavern wall closest to the facility and pouring all of its energy into a contraption on the opposite side.

Upon closer inspection, the contraption appears to be a flat, two-dimensional square grid containing **empty space** (`.`), **mirrors** (`/` and `\`), and **splitters** (`|` and `-`).

The contraption is aligned so that most of the beam bounces around the grid, but each tile on the grid converts some of the beam's light into **heat** to melt the rock in the cavern.

You note the layout of the contraption (your puzzle input). For example:

```
.|...\....
|.-.\.....
.....|-...
........|.
..........
.........\
..../.\\..
.-.-/..|..
.|....-|.\
..//.|....
```

The beam enters in the top-left corner from the left and heading to the **right**. Then, its behavior depends on what it encounters as it moves:

- If the beam encounters **empty space** (`.`), it continues in the same direction.
- If the beam encounters a **mirror** (`/` or `\`), the beam is **reflected** 90 degrees depending on the angle of the mirror. For instance, a rightward-moving beam that encounters a `/` mirror would continue **upward** in the mirror's column, while a rightward-moving beam that encounters a `\` mirror would continue **downward** from the mirror's column.
- If the beam encounters the **pointy end of a splitter** (`|` or `-`), the beam passes through the splitter as if the splitter were **empty space**. For instance, a rightward-moving beam that encounters a `-` splitter would continue in the same direction.
- If the beam encounters the **flat side of a splitter** (`|` or `-`), the beam is **split into two beams** going in each of the two directions the splitter's pointy ends are pointing. For instance, a rightward-moving beam that encounters a `|` splitter would split into two beams: one that continues **upward** from the splitter's column and one that continues **downward** from the splitter's column.

Beams do not interact with other beams; a tile can have many beams passing through it at the same time. A tile is **energized** if that tile has at least one beam pass through it, reflect in it, or split in it.

In the above example, here is how the beam of light bounces around the contraption:

```
>|<<<\....
|v-.\^....
.v...|->>>
.v...v^.|.
.v...v^...
.v...v^..\
.v../2\\..
<->-/vv|..
.|<<<2-|.\
.v//.|.v..
```

Beams are only shown on empty tiles; arrows indicate the direction of the beams. If a tile contains beams moving in multiple directions, the number of distinct directions is shown instead. Here is the same diagram but instead only showing whether a tile is **energized** (`#`) or not (`.`):

```
######....
.#...#....
.#...#####
.#...##...
.#...##...
.#...##...
.#..####..
########..
.#######..
.#...#.#..
```

Ultimately, in this example, `46` tiles become **energized**.

The light isn't energizing enough tiles to produce lava; to debug the contraption, you need to start by analyzing the current situation. With the beam starting in the top-left heading right, **how many tiles end up being energized?**

In [47]:
RIGHT = (0, 1)
LEFT = (0, -1)
UP = (-1, 0)
DOWN = (1, 0)

def handle_cell(lines, beam):
    new_location = [x+y for x,y in zip(beam[0], beam[1])]

    if not ((0 <= new_location[1] < len(lines[0])) and (0 <= new_location[0] < len(lines))):
        return None

    symbol = lines[new_location[0]][new_location[1]]
    if symbol == '.':
        return [[new_location, beam[1]]]

    if symbol == '|':
        if beam[1] in [UP, DOWN]:
            return [[new_location, beam[1]]]
        if beam[1] in [RIGHT, LEFT]:
            return [[new_location, UP], [new_location, DOWN]]

    if symbol == '-':
        if beam[1] in [RIGHT, LEFT]:
            return [[new_location, beam[1]]]
        if beam[1] in [UP, DOWN]:
            return [[new_location, RIGHT], [new_location, LEFT]]

    if symbol == '/':
        if beam[1] == RIGHT:
            return [[new_location, UP]]
        if beam[1] == LEFT:
            return [[new_location, DOWN]]
        if beam[1] == UP:
            return [[new_location, RIGHT]]
        if beam[1] == DOWN:
            return [[new_location, LEFT]]

    if symbol == '\\':
        if beam[1] == RIGHT:
            return [[new_location, DOWN]]
        if beam[1] == LEFT:
            return [[new_location, UP]]
        if beam[1] == UP:
            return [[new_location, LEFT]]
        if beam[1] == DOWN:
            return [[new_location, RIGHT]]

with open('day_16_input.txt', 'r') as input:
    lines = input.readlines()

start = [[0,-1], RIGHT]

active_beams = [start]
locations_seen = []

while len(active_beams) > 0:
    new_active_beams = []
    for beam in active_beams:
        next_cells = handle_cell(lines, beam)
        if next_cells is not None:
            for next_cell in next_cells:
                if (next_cell is not None) and not (next_cell in locations_seen):
                    new_active_beams.append(next_cell)
                if not (next_cell in locations_seen):
                    locations_seen.append(next_cell)
    active_beams = new_active_beams

uniques = []
for x in [location[0] for location in locations_seen]:
    if x not in uniques:
        uniques.append(x)

print(len(uniques))


6795


## --- Part Two ---

As you try to work out what might be wrong, the reindeer tugs on your shirt and leads you to a nearby control panel. There, a collection of buttons lets you align the contraption so that the beam enters from **any edge tile** and heading away from that edge. (You can choose either of two directions for the beam if it starts on a corner; for instance, if the beam starts in the bottom-right corner, it can start heading either left or upward.)

So, the beam could start on any tile in the top row (heading downward), any tile in the bottom row (heading upward), any tile in the leftmost column (heading right), or any tile in the rightmost column (heading left). To produce lava, you need to find the configuration that **energizes as many tiles as possible**.

In the above example, this can be achieved by starting the beam in the fourth tile from the left in the top row:

```
.|<2<\....
|v-v\^....
.v.v.|->>>
.v.v.v^.|.
.v.v.v^...
.v.v.v^..\
.v.v/2\\..
<-2-/vv|..
.|<<<2-|.\
.v//.|.v..
```

Using this configuration, **`51`** tiles are energized:

```
.#####....
.#.#.#....
.#.#.#####
.#.#.##...
.#.#.##...
.#.#.##...
.#.#####..
########..
.#######..
.#...#.#..
```

Find the initial beam configuration that energizes the largest number of tiles; **how many tiles are energized in that configuration?**

In [53]:
RIGHT = (0, 1)
LEFT = (0, -1)
UP = (-1, 0)
DOWN = (1, 0)

def handle_cell(lines, beam):
    new_location = [x+y for x,y in zip(beam[0], beam[1])]

    if not ((0 <= new_location[1] < len(lines[0])) and (0 <= new_location[0] < len(lines))):
        return None

    symbol = lines[new_location[0]][new_location[1]]
    if symbol == '.':
        return [[new_location, beam[1]]]

    if symbol == '|':
        if beam[1] in [UP, DOWN]:
            return [[new_location, beam[1]]]
        if beam[1] in [RIGHT, LEFT]:
            return [[new_location, UP], [new_location, DOWN]]

    if symbol == '-':
        if beam[1] in [RIGHT, LEFT]:
            return [[new_location, beam[1]]]
        if beam[1] in [UP, DOWN]:
            return [[new_location, RIGHT], [new_location, LEFT]]

    if symbol == '/':
        if beam[1] == RIGHT:
            return [[new_location, UP]]
        if beam[1] == LEFT:
            return [[new_location, DOWN]]
        if beam[1] == UP:
            return [[new_location, RIGHT]]
        if beam[1] == DOWN:
            return [[new_location, LEFT]]

    if symbol == '\\':
        if beam[1] == RIGHT:
            return [[new_location, DOWN]]
        if beam[1] == LEFT:
            return [[new_location, UP]]
        if beam[1] == UP:
            return [[new_location, LEFT]]
        if beam[1] == DOWN:
            return [[new_location, RIGHT]]

with open('day_16_input.txt', 'r') as input:
    lines = input.readlines()

max_energized = 0
for direction, edges in zip([UP, DOWN, RIGHT, LEFT], [len(lines[0]), len(lines[0]), len(lines), len(lines)]):
    for edge in range(edges):

        if direction == RIGHT:
            start = [[edge,-1], RIGHT]
        if direction == LEFT:
            start = [[edge,len(lines[0])], LEFT]
        if direction == DOWN:
            start = [[-1,edge], DOWN]
        if direction == UP:
            start = [[len(lines),edge], UP]

        active_beams = [start]
        locations_seen = []

        while len(active_beams) > 0:
            new_active_beams = []
            for beam in active_beams:
                next_cells = handle_cell(lines, beam)
                if next_cells is not None:
                    for next_cell in next_cells:
                        if (next_cell is not None) and not (next_cell in locations_seen):
                            new_active_beams.append(next_cell)
                        if not (next_cell in locations_seen):
                            locations_seen.append(next_cell)
            active_beams = new_active_beams

        uniques = []
        for x in [location[0] for location in locations_seen]:
            if x not in uniques:
                uniques.append(x)

        if len(uniques) > max_energized:
            max_energized = len(uniques)

print(max_energized)

7154
